# Cell cell communication

In this notebook, we show how to use Thor's built in module to study cell-cell communication infer cell-level spatial transcriptome on a sample from human heart failure patients with myocardial infarction.

For installation of Thor, please refer to [this installation guide](../installation.rst).

In [ ]:
import sys
import os
import logging
import datetime
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

now = datetime.datetime.now()

print(f"Current Time: {now}")

here = os.getcwd()
print(f"Current Directory: {here}")

In [2]:
import commot as ct
import thor
from thor.analysis.ccc import split_pathways, run_commot, plot_commot

%config InlineBackend.figure_format = 'retina'

## Load ligand-receptor database

In [ ]:
cc = ct.pp.ligand_receptor_database(database='CellChat', species='human')
cc

In [ ]:
pathways_dict = split_pathways(cc, name_col_index=2)
pathways_dict['VEGF']

## Human Heart (spot level)

The heart failure tissue sample is downloaded from [the study](https://doi.org/10.1038/s41586-022-05060-x) in a Nature paper "Spatial multi-omic map of human myocardial infarction".

In [5]:
sn = 'GT_IZ_P9_rep2'
ad_spot_path = f"{sn}_adata_spots_filtered_ref.h5ad"
ad = sc.read_h5ad(ad_spot_path)

In [6]:
from thor.utils import convert_pixel_to_micron_visium

microns_per_pixel = convert_pixel_to_micron_visium(ad, res='fullres', spotDiameterinMicron=65)
distance = 100 # distance threshold in microns
distance_pixel = distance / microns_per_pixel

### Use Thor's API: run_commot for data preprocessing and run COMMOT

In [7]:
database = 'CellChat'
pathway = 'VEGF'

ad_comm = run_commot(
    ad,
    database_name=database, 
    df_ligrec=pathways_dict[pathway], 
    dis_thr=distance_pixel, 
    heteromeric=True,
    pathway_sum=True)

### We can now check the directions of VEGF signals via a streamplot (spot level)

In [ ]:
plot_commot(ad_comm, database_name=database, pathway_name=pathway, plot_method='stream', background_legend=True,
    scale=0.5, ndsize=8, grid_density=0.4, summary='sender', background='image', clustering='leiden', cmap='Alphabet',
    normalize_v=True, normalize_v_quantile=0.995)

### With the original spot data, VEGF signals are observed crossing a vessel region

In [ ]:
plot_commot(ad_comm, region=(4000, 5800, 3000, 5500), database_name=database, pathway_name=pathway, plot_method='stream', background_legend=True,
    scale=0.5, ndsize=4, grid_density=0.4, summary='sender', background='image', clustering='leiden', cmap='Alphabet',
    normalize_v=True, normalize_v_quantile=0.995)

## Human Heart (cell level based on Thor)

Subsample 2 cells in every spot for efficiency

In [ ]:
ad_thor = sc.read_h5ad("GT_IZ_P9_rep2_adata_cells.h5ad")
ad_thor

In [ ]:
spot_barcodes = ad_thor.obs.spot_barcodes.drop_duplicates()
spot_barcodes = list(map(lambda x: str(x)+'-1', spot_barcodes)) + list(spot_barcodes)
ad_subsampled = ad_thor[ad_thor.obs.index.isin(spot_barcodes), :].copy()
ad_subsampled

### Use Thor's API: run_commot for data preprocessing and run COMMOT

In [12]:
ad_thor_comm = run_commot(
    ad_subsampled,
    database_name=database, 
    df_ligrec=pathways_dict[pathway],
    dis_thr=distance_pixel,
    heteromeric=True,
    pathway_sum=True)

### We can now check the directions of VEGF signals via a streamplot (cell level)

In [ ]:
plot_commot(ad_thor_comm, database_name=database, pathway_name=pathway, plot_method='stream', background_legend=True,
    scale=0.5, ndsize=4, grid_density=0.2, summary='sender', background='image', clustering='leiden', cmap='Alphabet',
    normalize_v=True, normalize_v_quantile=0.995)

### With Thor inferred data, VEGF signals are observed initialized from a vessel region, which makes more sense

In [ ]:
plot_commot(ad_thor_comm, region=(4000, 5800, 3000, 5500), database_name=database, pathway_name=pathway, plot_method='stream', background_legend=True,
    scale=0.5, ndsize=4, grid_density=0.2, summary='sender', background='image', clustering='leiden', cmap='Alphabet',
    normalize_v=True, normalize_v_quantile=0.995)

### DEG along signaling pathways

In [ ]:
ad_thor_comm.layers['counts'] = np.expm1(ad_thor_comm.X)

df_deg, df_yhat = ct.tl.communication_deg_detection(
    ad_thor_comm,
    database_name=database, 
    pathway_name=pathway, 
    summary='receiver')

In [ ]:
df_deg_clus, df_yhat_clus = ct.tl.communication_deg_clustering(df_deg, df_yhat, deg_clustering_res=0.1)

top_de_genes_VEGF = ct.pl.plot_communication_dependent_genes(
    df_deg_clus, 
    df_yhat_clus, 
    top_ngene_per_cluster=5,
    filename='./heatmap_deg_VEGF.pdf', 
    font_scale=1.2, 
    return_genes=True)

In [ ]:
def plot_genes(adata_dis500, pathway, genes, database=None):
    X_sc = adata_dis500.obsm['spatial']
    fig, ax = plt.subplots(1,3, figsize=(15,4))
    colors = adata_dis500.obsm[f'commot-{database}-sum-sender'][f's-{pathway}'].values
    idx = np.argsort(colors)
    ax[0].scatter(X_sc[idx,0],X_sc[idx,1], c=colors[idx], cmap='coolwarm', s=2)
    colors = adata_dis500[:,genes[0]].X.toarray().flatten()
    idx = np.argsort(colors)
    ax[1].scatter(X_sc[idx,0],X_sc[idx,1], c=colors[idx], cmap='coolwarm', s=2)
    colors = adata_dis500[:,genes[1]].X.toarray().flatten()
    idx = np.argsort(colors)
    ax[2].scatter(X_sc[idx,0],X_sc[idx,1], c=colors[idx], cmap='coolwarm', s=2)
    ax[0].set_title('Amount of received signal')
    ax[1].set_title(f'An example negative DE gene ({genes[0]})')
    ax[2].set_title(f'An example positive DE gene ({genes[1]})')
    ax[0].invert_yaxis()
    ax[1].invert_yaxis()
    ax[2].invert_yaxis()
    plt.show()


plot_genes(ad_thor_comm, pathway, ['MELTF', 'IGFBP5'], database=database)